In [2]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch import nn
# import torch.nn.utils.prune as prune
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
from torch import Tensor
from torch.nn.parameter import Parameter

from torchvision import datasets, transforms
from torch.utils.data import DataLoader

import math
import warnings

RANDOM_SEED = 42
LEARNING_RATE = 0.001
BATCH_SIZE = 32
N_EPOCHS = 15

IMG_SIZE = 32
N_CLASSES = 10
DEVICE = 'cpu'

In [3]:
def train(train_loader, model, criterion, optimizer, device):
    '''
    Function for the training step of the training loop
    '''

    model.train()
    running_loss = 0
    
    for X, y_true in train_loader:

        optimizer.zero_grad()
        
        X = X.to(device)
        y_true = y_true.to(device)
    
        # Forward pass
        y_hat, _ = model(X) 
        loss = criterion(y_hat, y_true) 
        running_loss += loss.item() * X.size(0)

        # Backward pass
        loss.backward()
        optimizer.step()
        
    epoch_loss = running_loss / len(train_loader.dataset)
    return model, optimizer, epoch_loss

def training_loop(model, criterion, optimizer, train_loader, valid_loader, epochs, device, print_every=1):
    '''
    Function defining the entire training loop
    '''
    
    # set objects for storing metrics
    best_loss = 1e10
    train_losses = []
    valid_losses = []
 
    # Train model
    for epoch in range(0, epochs):

        # training
        model, optimizer, train_loss = train(train_loader, model, criterion, optimizer, device)
        train_losses.append(train_loss)

        # validation
        with torch.no_grad():
            model, valid_loss = validate(valid_loader, model, criterion, device)
            valid_losses.append(valid_loss)

#         if epoch % print_every == (print_every - 1):
            
#             train_acc = get_accuracy(model, train_loader, device=device)
#             valid_acc = get_accuracy(model, valid_loader, device=device)
                
#             print(f'{datetime.now().time().replace(microsecond=0)} --- '
#                   f'Epoch: {epoch}\t'
#                   f'Train loss: {train_loss:.4f}\t'
#                   f'Valid loss: {valid_loss:.4f}\t'
#                   f'Train accuracy: {100 * train_acc:.2f}\t'
#                   f'Valid accuracy: {100 * valid_acc:.2f}')

#     plot_losses(train_losses, valid_losses)
    
    return model, optimizer, (train_losses, valid_losses)

def validate(valid_loader, model, criterion, device):
    '''
    Function for the validation step of the training loop
    '''
   
    model.eval()
    running_loss = 0
    
    for X, y_true in valid_loader:
    
        X = X.to(device)
        y_true = y_true.to(device)

        # Forward pass and record loss
        y_hat, _ = model(X) 
        loss = criterion(y_hat, y_true) 
        running_loss += loss.item() * X.size(0)

    epoch_loss = running_loss / len(valid_loader.dataset)
        
    return model, epoch_loss

In [4]:
# define transforms
transforms = transforms.Compose([transforms.Resize((32, 32)),
                                 transforms.ToTensor()])

# download and create datasets
train_dataset = datasets.MNIST(root='mnist_data', 
                               train=True, 
                               transform=transforms,
                               download=True)

valid_dataset = datasets.MNIST(root='mnist_data', 
                               train=False, 
                               transform=transforms)

# define the data loaders
train_loader = DataLoader(dataset=train_dataset, 
                          batch_size=BATCH_SIZE, 
                          shuffle=True)

valid_loader = DataLoader(dataset=valid_dataset, 
                          batch_size=BATCH_SIZE, 
                          shuffle=False)

class LeNet5(nn.Module):

    def __init__(self, n_classes):
        super(LeNet5, self).__init__()
        
        self.feature_extractor = nn.Sequential(            
            nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5, stride=1,bias=False),
            nn.Tanh(),
            nn.AvgPool2d(kernel_size=2),
            nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5, stride=1,bias=False),
            nn.Tanh(),
            nn.AvgPool2d(kernel_size=2),
            nn.Conv2d(in_channels=16, out_channels=120, kernel_size=5, stride=1,bias=False),
            nn.Tanh()
        )

        self.classifier = nn.Sequential(
            nn.Linear(in_features=120, out_features=84,bias=False),
            nn.Tanh(),
            nn.Linear(in_features=84, out_features=n_classes,bias=False),
        )


    def forward(self, x):
        x = self.feature_extractor(x)
        x = torch.flatten(x, 1)
        logits = self.classifier(x)
        probs = F.softmax(logits, dim=1)
        return logits, probs
    
torch.manual_seed(RANDOM_SEED)

model = LeNet5(N_CLASSES).to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
criterion = nn.CrossEntropyLoss()
    
model, optimizer, _ = training_loop(model, criterion, optimizer, train_loader, valid_loader, N_EPOCHS, DEVICE)

# def get_datasets(*args, **kwargs):
#     transform = transforms.Compose(
#         [
#             transforms.ToTensor()
# #             transforms.Normalize((0.1307,), (0.3081,))
#         ]
#     )

#     trainset = torchvision.datasets.MNIST(train=True, transform=transform, *args, **kwargs)
#     testset = torchvision.datasets.MNIST(train=False, transform=transform, *args, **kwargs)
#     return trainset, testset

# def get_dataloaders(trainset, testset, batch_size=100, num_worker=4):
#     trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=num_worker)
#     testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=num_worker)

#     return trainloader, testloader

# LeNET = LeNet5(10)
# criterion = nn.CrossEntropyLoss()
# lr_ = 0.002
# optimizer = optim.SGD(LeNET.feature_extractor.parameters(), lr=lr_)
# optimizer2 = optim.SGD(LeNET.classifier.parameters(), lr=lr_)


# trainset, testset = get_datasets(root='./data', download=True)
# trainloader, testloader = get_dataloaders(trainset, testset, batch_size=100, num_worker=16)
# loss_ = np.zeros(15)
# gradient = np.zeros((4,15))

# for epoch in range(20):  # loop over the dataset multiple times
#     running_loss = 0.0
#     for i, data in enumerate(trainloader, 0):
#         # get the inputs; data is a list of [inputs, labels]
#         inputs, labels = data

#         # zero the parameter gradients
#         optimizer.zero_grad()
#         optimizer2.zero_grad()
#         outputs = LeNET(inputs)

        
#         loss = criterion(outputs, labels)
#         loss.backward()

#         optimizer.step()
#         optimizer2.step

#         running_loss += loss.item()

    
# print('Finished Training')

# PATH = './mnist_LeNET.pth'
# torch.save(LeNET.state_dict(), PATH)

In [5]:
def get_datasets(*args, **kwargs):
    transform = transforms.Compose(
        [
            transforms.ToTensor()
#             transforms.Normalize((0.1307,), (0.3081,))
        ]
    )

    trainset = torchvision.datasets.MNIST(train=True, transform=transform, *args, **kwargs)
    testset = torchvision.datasets.MNIST(train=False, transform=transform, *args, **kwargs)
    return trainset, testset

def get_dataloaders(trainset, testset, batch_size=100, num_worker=4):
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=num_worker)
    testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=num_worker)

    return trainloader, testloader

trainset, testset = get_datasets(root='./data', download=True)
trainloader, testloader = get_dataloaders(trainset, testset, batch_size=100, num_worker=16)

correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

AttributeError: 'Compose' object has no attribute 'Compose'